In [31]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [32]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-8h53s569
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-8h53s569
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 781ff5b76ba6c4c2d80dcbbec9983e147613cc71
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [33]:
%load_ext nvcc4jupyter

The nvcc4jupyter extension is already loaded. To reload it, use:
  %reload_ext nvcc4jupyter


**Kernal1  (Each thread produces one output matrix element)**

In [34]:

%%writefile K1_2_31_2_33.cu
//Norhan_Reda_Abdelwahed_2_31
//Hoda_Gamal_Hamouda_2_33
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <assert.h>
#include <iostream>
#include <fstream>
#include <vector>
using namespace std;
#define N 800
// Kernel definition
__global__ void MatAdd(float *A, float *B, float *C, int rows, int cols)
{
    int j = blockIdx.x * blockDim.x + threadIdx.x;
    int i = blockIdx.y * blockDim.y + threadIdx.y;
    if (i < rows && j < cols)
        C[i * cols + j] = A[i * cols + j] + B[i * cols + j];

}

int main(int argc, char *argv[])
{


    // Specify the file path
    std::string filePath = argv[1];

    // Open the file for reading
    std::ifstream inputFile(filePath);
    std::string outputFilePath = argv[2];
    // Check if the file is opened successfully
    if (!inputFile.is_open()) {
        std::cerr << "Error opening file: " << filePath << std::endl;
        return 1; // Exit with an error code
    }

    // Open the output file for writing
    std::ofstream outputFile(outputFilePath);
      // Check if the output file is opened successfully
    if (!outputFile.is_open()) {
        std::cerr << "Error opening output file: " << outputFilePath << std::endl;
        inputFile.close(); // Close the input file before exiting
        return 1; // Exit with an error code
    }


    // Read the number of test cases
    int numTestCases;
    int rows ;
   int cols;
    inputFile >> numTestCases;

    // Process each test case
    for (int testCase = 1; testCase <= numTestCases; ++testCase) {
        std::cout << "Test Case " << testCase << ":" << std::endl;

        // Read the number of rows and columns for the current test case

        inputFile >> rows >> cols;

        cout<<rows<<","<<cols<<endl;
 float *h_A, *h_B, *h_C;
    float *d_A, *d_B, *d_C;

    // Allocate host memory
    h_A = (float*)malloc(sizeof(float) * rows * cols);
    h_B =(float*)malloc(sizeof(float) * rows * cols);
    h_C = (float*)malloc(sizeof(float) * rows * cols);

    // Populate matrices h_A and h_B

    // Allocate device memory
    cudaMalloc((void**)&d_A, rows * cols * sizeof(float));
    cudaMalloc((void**)&d_B, rows * cols * sizeof(float));
    cudaMalloc((void**)&d_C, rows * cols * sizeof(float));




        // Read matrix1 elements
        for (int i = 0; i < rows; ++i) {
            for (int j = 0; j < cols; ++j) {
                inputFile >> h_A[i * cols + j];
            }
        }

        // Read matrix2 elements
        for (int i = 0; i < rows; ++i) {
            for (int j = 0; j < cols; ++j) {
                inputFile >> h_B[i * cols + j];
            }
        }

        // Print matrices for the current test case (you can replace this with your specific logic)
        std::cout << "Matrix1:" << std::endl;
        for (int i = 0; i < rows; ++i) {
            for (int j = 0; j < cols; ++j) {
                std::cout << h_A[i * cols + j] << " ";
            }
            std::cout << std::endl;
        }

        std::cout << "Matrix2:" << std::endl;
        for (int i = 0; i < rows; ++i) {
            for (int j = 0; j < cols; ++j) {
                std::cout << h_B[i * cols + j] << " ";
            }
            std::cout << std::endl;
        }

      // Copy data from host to device
    cudaMemcpy(d_A, h_A, rows * cols * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, h_B, rows * cols * sizeof(float), cudaMemcpyHostToDevice);



    //...
    // Kernel invocation with one block of N * N * 1 threads
    dim3 threadsPerBlock(32, 32);
    dim3 numBlocks((cols-1) / threadsPerBlock.x+1,(rows-1) / threadsPerBlock.y+1);
    MatAdd<<<numBlocks, threadsPerBlock>>>(d_A, d_B, d_C,rows,cols);
    //...
    // Copy the result matrix C from device to host
    cudaMemcpy(h_C, d_C, rows * cols * sizeof(float), cudaMemcpyDeviceToHost);

    printf("h_C[0] = %f\n", h_C[0]);
     printf("h_C[last] = %f\n", h_C[rows*cols-1]);
    printf("PASSED\n");
    // Write matrices for the current test case to the output file

        for (int i = 0; i < rows; ++i) {
            for (int j = 0; j < cols; ++j) {
                outputFile << h_C[i * cols + j] << " ";
            }
            outputFile << std::endl;
        }

    // Free device memory
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);

    // Free host memory
    free(h_A);
    free(h_B);
    free(h_C);

    }

    // Close the file
    inputFile.close();
    outputFile.close();

   return 0;
}


Overwriting K1_2_31_2_33.cu


In [35]:
!chmod +x K1_2_31_2_33.cu
!nvcc K1_2_31_2_33.cu -o K1
!./K1 "Q1-sample-testcases-template-file.txt" "Q1-sample-output-file.txt"

Test Case 1:
3,4
Matrix1:
1.1 2.4 3.5 4.6 
5.9 6.2 7.3 8.8 
9.7 4.8 3.1 2.3 
Matrix2:
3.4 5.7 8.9 3.2 
2.4 4.5 6.3 3.9 
3.2 2.3 4.8 6.3 
h_C[0] = 4.500000
h_C[last] = 8.600000
PASSED
Test Case 2:
2,2
Matrix1:
1.2 4.5 
2.3 7.8 
Matrix2:
2.7 6.3 
2.9 6.4 
h_C[0] = 3.900000
h_C[last] = 14.200001
PASSED


In [36]:
!nvprof ./K1 "Q1-sample-testcases-template-file.txt" "Q1-sample-output-file.txt"

Test Case 1:
3,4
==4108== NVPROF is profiling process 4108, command: ./K1 Q1-sample-testcases-template-file.txt Q1-sample-output-file.txt
Matrix1:
1.1 2.4 3.5 4.6 
5.9 6.2 7.3 8.8 
9.7 4.8 3.1 2.3 
Matrix2:
3.4 5.7 8.9 3.2 
2.4 4.5 6.3 3.9 
3.2 2.3 4.8 6.3 
h_C[0] = 4.500000
h_C[last] = 8.600000
PASSED
Test Case 2:
2,2
Matrix1:
1.2 4.5 
2.3 7.8 
Matrix2:
2.7 6.3 
2.9 6.4 
h_C[0] = 3.900000
h_C[last] = 14.200001
PASSED
==4108== Profiling application: ./K1 Q1-sample-testcases-template-file.txt Q1-sample-output-file.txt
==4108== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   48.52%  6.8160us         2  3.4080us  3.3590us  3.4570us  MatAdd(float*, float*, float*, int, int)
                   26.87%  3.7750us         2  1.8870us  1.6630us  2.1120us  [CUDA memcpy DtoH]
                   24.61%  3.4570us         4     864ns     672ns  1.1210us  [CUDA memcpy HtoD]
      API calls:   99.58%  166.44ms         6  27.741ms  3


**Kernal 2 ( Each thread produces one output matrix row)**

In [37]:

%%writefile K2_2_31_2_33.cu
//Norhan_Reda_Abdelwahed_2_31
//Hoda_Gamal_Hamouda_2_33
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <assert.h>
#include <iostream>
#include <fstream>
#include <vector>
using namespace std;
#define N 800
// Kernel definition
__global__ void MatAdd(float *A, float *B, float *C, int rows, int cols)
{
    int i = blockIdx.y * blockDim.y + threadIdx.y;
    if (i < rows){
   for (int j = 0; j < cols; j++) {

           C[i * cols + j] = A[i * cols + j] + B[i * cols + j];
    }
    }


}

int main(int argc, char *argv[])
{


    // Specify the file path
    std::string filePath = argv[1];

    // Open the file for reading
    std::ifstream inputFile(filePath);
    std::string outputFilePath = argv[2];
    // Check if the file is opened successfully
    if (!inputFile.is_open()) {
        std::cerr << "Error opening file: " << filePath << std::endl;
        return 1; // Exit with an error code
    }

    // Open the output file for writing
    std::ofstream outputFile(outputFilePath);
      // Check if the output file is opened successfully
    if (!outputFile.is_open()) {
        std::cerr << "Error opening output file: " << outputFilePath << std::endl;
        inputFile.close(); // Close the input file before exiting
        return 1; // Exit with an error code
    }


    // Read the number of test cases
    int numTestCases;
    int rows ;
   int cols;
    inputFile >> numTestCases;

    // Process each test case
    for (int testCase = 1; testCase <= numTestCases; ++testCase) {
        std::cout << "Test Case " << testCase << ":" << std::endl;

        // Read the number of rows and columns for the current test case

        inputFile >> rows >> cols;

        cout<<rows<<","<<cols<<endl;
 float *h_A, *h_B, *h_C;
    float *d_A, *d_B, *d_C;

    // Allocate host memory
    h_A = (float*)malloc(sizeof(float) * rows * cols);
    h_B =(float*)malloc(sizeof(float) * rows * cols);
    h_C = (float*)malloc(sizeof(float) * rows * cols);

    // Populate matrices h_A and h_B

    // Allocate device memory
    cudaMalloc((void**)&d_A, rows * cols * sizeof(float));
    cudaMalloc((void**)&d_B, rows * cols * sizeof(float));
    cudaMalloc((void**)&d_C, rows * cols * sizeof(float));




        // Read matrix1 elements
        for (int i = 0; i < rows; ++i) {
            for (int j = 0; j < cols; ++j) {
                inputFile >> h_A[i * cols + j];
            }
        }

        // Read matrix2 elements
        for (int i = 0; i < rows; ++i) {
            for (int j = 0; j < cols; ++j) {
                inputFile >> h_B[i * cols + j];
            }
        }

        // Print matrices for the current test case (you can replace this with your specific logic)
        std::cout << "Matrix1:" << std::endl;
        for (int i = 0; i < rows; ++i) {
            for (int j = 0; j < cols; ++j) {
                std::cout << h_A[i * cols + j] << " ";
            }
            std::cout << std::endl;
        }

        std::cout << "Matrix2:" << std::endl;
        for (int i = 0; i < rows; ++i) {
            for (int j = 0; j < cols; ++j) {
                std::cout << h_B[i * cols + j] << " ";
            }
            std::cout << std::endl;
        }

      // Copy data from host to device
    cudaMemcpy(d_A, h_A, rows * cols * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, h_B, rows * cols * sizeof(float), cudaMemcpyHostToDevice);



    //...
    // Kernel invocation with one block of N * N * 1 threads
    dim3 threadsPerBlock(32, 32);
    dim3 numBlocks((cols-1) / threadsPerBlock.x+1,(rows-1) / threadsPerBlock.y+1);
    MatAdd<<<numBlocks, threadsPerBlock>>>(d_A, d_B, d_C,rows,cols);
    //...
    // Copy the result matrix C from device to host
    cudaMemcpy(h_C, d_C, rows * cols * sizeof(float), cudaMemcpyDeviceToHost);

    printf("h_C[0] = %f\n", h_C[0]);
    printf("PASSED\n");
    // Write matrices for the current test case to the output file

        for (int i = 0; i < rows; ++i) {
            for (int j = 0; j < cols; ++j) {
                outputFile << h_C[i * cols + j] << " ";
            }
            outputFile << std::endl;
        }

    // Free device memory
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);

    // Free host memory
    free(h_A);
    free(h_B);
    free(h_C);

    }

    // Close the file
    inputFile.close();
    outputFile.close();

   return 0;
}


Overwriting K2_2_31_2_33.cu


In [38]:
!chmod +x K2_2_31_2_33.cu
!nvcc K2_2_31_2_33.cu -o K2
!./K2 "Q1-sample-testcases-template-file.txt" "Q1-sample-output2-file.txt"

Test Case 1:
3,4
Matrix1:
1.1 2.4 3.5 4.6 
5.9 6.2 7.3 8.8 
9.7 4.8 3.1 2.3 
Matrix2:
3.4 5.7 8.9 3.2 
2.4 4.5 6.3 3.9 
3.2 2.3 4.8 6.3 
h_C[0] = 4.500000
PASSED
Test Case 2:
2,2
Matrix1:
1.2 4.5 
2.3 7.8 
Matrix2:
2.7 6.3 
2.9 6.4 
h_C[0] = 3.900000
PASSED


In [39]:
!nvprof ./K2 "Q1-sample-testcases-template-file.txt" "Q1-sample-output2-file.txt"

Test Case 1:
3,4
==4186== NVPROF is profiling process 4186, command: ./K2 Q1-sample-testcases-template-file.txt Q1-sample-output2-file.txt
Matrix1:
1.1 2.4 3.5 4.6 
5.9 6.2 7.3 8.8 
9.7 4.8 3.1 2.3 
Matrix2:
3.4 5.7 8.9 3.2 
2.4 4.5 6.3 3.9 
3.2 2.3 4.8 6.3 
h_C[0] = 4.500000
PASSED
Test Case 2:
2,2
Matrix1:
1.2 4.5 
2.3 7.8 
Matrix2:
2.7 6.3 
2.9 6.4 
h_C[0] = 3.900000
PASSED
==4186== Profiling application: ./K2 Q1-sample-testcases-template-file.txt Q1-sample-output2-file.txt
==4186== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   53.20%  8.2550us         2  4.1270us  3.6800us  4.5750us  MatAdd(float*, float*, float*, int, int)
                   24.33%  3.7750us         2  1.8870us  1.6630us  2.1120us  [CUDA memcpy DtoH]
                   22.48%  3.4880us         4     872ns     672ns  1.1520us  [CUDA memcpy HtoD]
      API calls:   99.56%  173.21ms         6  28.869ms  2.9200us  173.07ms  cudaMalloc
           

**Kernal 3 (Each thread produces one output matrix column)**

In [40]:

%%writefile K3_2_31_2_33.cu
//Norhan_Reda_Abdelwahed_2_31
//Hoda_Gamal_Hamouda_2_33
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <assert.h>
#include <iostream>
#include <fstream>
#include <vector>
using namespace std;
#define N 800
// Kernel definition
__global__ void MatAdd(float *A, float *B, float *C, int rows, int cols)
{

    int j = blockIdx.x * blockDim.x + threadIdx.x;
     if (j < cols) {
     for (int i = 0; i < rows; i++) {
             C[i * cols + j] = A[i * cols + j] + B[i * cols + j];
    }
     }

}

int main(int argc, char *argv[])
{


    // Specify the file path
    std::string filePath = argv[1];

    // Open the file for reading
    std::ifstream inputFile(filePath);
    std::string outputFilePath = argv[2];
    // Check if the file is opened successfully
    if (!inputFile.is_open()) {
        std::cerr << "Error opening file: " << filePath << std::endl;
        return 1; // Exit with an error code
    }

    // Open the output file for writing
    std::ofstream outputFile(outputFilePath);
      // Check if the output file is opened successfully
    if (!outputFile.is_open()) {
        std::cerr << "Error opening output file: " << outputFilePath << std::endl;
        inputFile.close(); // Close the input file before exiting
        return 1; // Exit with an error code
    }


    // Read the number of test cases
    int numTestCases;
    int rows ;
   int cols;
    inputFile >> numTestCases;

    // Process each test case
    for (int testCase = 1; testCase <= numTestCases; ++testCase) {
        std::cout << "Test Case " << testCase << ":" << std::endl;

        // Read the number of rows and columns for the current test case

        inputFile >> rows >> cols;

        cout<<rows<<","<<cols<<endl;
 float *h_A, *h_B, *h_C;
    float *d_A, *d_B, *d_C;

    // Allocate host memory
    h_A = (float*)malloc(sizeof(float) * rows * cols);
    h_B =(float*)malloc(sizeof(float) * rows * cols);
    h_C = (float*)malloc(sizeof(float) * rows * cols);

    // Populate matrices h_A and h_B

    // Allocate device memory
    cudaMalloc((void**)&d_A, rows * cols * sizeof(float));
    cudaMalloc((void**)&d_B, rows * cols * sizeof(float));
    cudaMalloc((void**)&d_C, rows * cols * sizeof(float));




        // Read matrix1 elements
        for (int i = 0; i < rows; ++i) {
            for (int j = 0; j < cols; ++j) {
                inputFile >> h_A[i * cols + j];
            }
        }

        // Read matrix2 elements
        for (int i = 0; i < rows; ++i) {
            for (int j = 0; j < cols; ++j) {
                inputFile >> h_B[i * cols + j];
            }
        }

        // Print matrices for the current test case (you can replace this with your specific logic)
        std::cout << "Matrix1:" << std::endl;
        for (int i = 0; i < rows; ++i) {
            for (int j = 0; j < cols; ++j) {
                std::cout << h_A[i * cols + j] << " ";
            }
            std::cout << std::endl;
        }

        std::cout << "Matrix2:" << std::endl;
        for (int i = 0; i < rows; ++i) {
            for (int j = 0; j < cols; ++j) {
                std::cout << h_B[i * cols + j] << " ";
            }
            std::cout << std::endl;
        }

      // Copy data from host to device
    cudaMemcpy(d_A, h_A, rows * cols * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, h_B, rows * cols * sizeof(float), cudaMemcpyHostToDevice);



    //...
    // Kernel invocation with one block of N * N * 1 threads
    dim3 threadsPerBlock(32, 32);
    dim3 numBlocks((cols-1) / threadsPerBlock.x+1,(rows-1) / threadsPerBlock.y+1);
    MatAdd<<<numBlocks, threadsPerBlock>>>(d_A, d_B, d_C,rows,cols);
    //...
    // Copy the result matrix C from device to host
    cudaMemcpy(h_C, d_C, rows * cols * sizeof(float), cudaMemcpyDeviceToHost);

    printf("h_C[0] = %f\n", h_C[0]);
    printf("PASSED\n");
    // Write matrices for the current test case to the output file

        for (int i = 0; i < rows; ++i) {
            for (int j = 0; j < cols; ++j) {
                outputFile << h_C[i * cols + j] << " ";
            }
            outputFile << std::endl;
        }

    // Free device memory
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);

    // Free host memory
    free(h_A);
    free(h_B);
    free(h_C);

    }

    // Close the file
    inputFile.close();
    outputFile.close();

   return 0;
}


Overwriting K3_2_31_2_33.cu


In [41]:
!chmod +x K3_2_31_2_33.cu
!nvcc K3_2_31_2_33.cu -o K3
!./K3 "Q1-sample-testcases-template-file.txt" "Q1-sample-output3-file.txt"

Test Case 1:
3,4
Matrix1:
1.1 2.4 3.5 4.6 
5.9 6.2 7.3 8.8 
9.7 4.8 3.1 2.3 
Matrix2:
3.4 5.7 8.9 3.2 
2.4 4.5 6.3 3.9 
3.2 2.3 4.8 6.3 
h_C[0] = 4.500000
PASSED
Test Case 2:
2,2
Matrix1:
1.2 4.5 
2.3 7.8 
Matrix2:
2.7 6.3 
2.9 6.4 
h_C[0] = 3.900000
PASSED


In [42]:
!nvprof ./K3  "Q1-sample-testcases-template-file.txt" "Q1-sample-output3-file.txt"

Test Case 1:
3,4
==4252== NVPROF is profiling process 4252, command: ./K3 Q1-sample-testcases-template-file.txt Q1-sample-output3-file.txt
Matrix1:
1.1 2.4 3.5 4.6 
5.9 6.2 7.3 8.8 
9.7 4.8 3.1 2.3 
Matrix2:
3.4 5.7 8.9 3.2 
2.4 4.5 6.3 3.9 
3.2 2.3 4.8 6.3 
h_C[0] = 4.500000
PASSED
Test Case 2:
2,2
Matrix1:
1.2 4.5 
2.3 7.8 
Matrix2:
2.7 6.3 
2.9 6.4 
h_C[0] = 3.900000
PASSED
==4252== Profiling application: ./K3 Q1-sample-testcases-template-file.txt Q1-sample-output3-file.txt
==4252== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   53.98%  8.4480us         2  4.2240us  3.9360us  4.5120us  MatAdd(float*, float*, float*, int, int)
                   24.34%  3.8090us         2  1.9040us  1.6650us  2.1440us  [CUDA memcpy DtoH]
                   21.68%  3.3920us         4     848ns     640ns  1.1200us  [CUDA memcpy HtoD]
      API calls:   99.59%  171.18ms         6  28.531ms  2.8810us  171.05ms  cudaMalloc
           

**Question 2 (a matrix vector multiplication kernel)**

In [43]:

%%writefile Q2_2_31_2_33.cu
//Norhan_Reda_Abdelwahed_2_31
//Hoda_Gamal_Hamouda_2_33
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <assert.h>
#include <iostream>
#include <fstream>
#include <vector>
using namespace std;
#define N 800
// Kernel definition
__global__ void MatMul(float *A, float *B, float *C, int rows, int cols)
{
    int row = blockIdx.y * blockDim.y + threadIdx.y;

    if (row < rows) {
        float element = 0.0f;
        for (int col = 0; col < cols; col++) {
            element += A[row * cols + col] * B[col];
        }
        C[row] = element;
    }

}

int main(int argc, char *argv[])
{


    // Specify the file path
    std::string filePath = argv[1];

    // Open the file for reading
    std::ifstream inputFile(filePath);
    std::string outputFilePath = argv[2];
    // Check if the file is opened successfully
    if (!inputFile.is_open()) {
        std::cerr << "Error opening file: " << filePath << std::endl;
        return 1; // Exit with an error code
    }

    // Open the output file for writing
    std::ofstream outputFile(outputFilePath);
      // Check if the output file is opened successfully
    if (!outputFile.is_open()) {
        std::cerr << "Error opening output file: " << outputFilePath << std::endl;
        inputFile.close(); // Close the input file before exiting
        return 1; // Exit with an error code
    }


    // Read the number of test cases
    int numTestCases;
    int rows ;
    int cols;
    inputFile >> numTestCases;

    // Process each test case
    for (int testCase = 1; testCase <= numTestCases; ++testCase) {
        std::cout << "Test Case " << testCase << ":" << std::endl;

        // Read the number of rows and columns for the current test case

        inputFile >> rows >> cols;

        cout<<rows<<","<<cols<<endl;
    float *h_A, *h_B, *h_C;
    float *d_A, *d_B, *d_C;

    // Allocate host memory
    h_A = (float*)malloc(sizeof(float) * rows * cols);
    h_B =(float*)malloc(sizeof(float) * cols *1 );
    h_C = (float*)malloc(sizeof(float) * rows * 1);

    // Populate matrices h_A and h_B

    // Allocate device memory
    cudaMalloc((void**)&d_A, rows * cols * sizeof(float));
    cudaMalloc((void**)&d_B, cols * 1 * sizeof(float));
    cudaMalloc((void**)&d_C, rows * 1 * sizeof(float));




        // Read matrix1 elements
        for (int i = 0; i < rows; ++i) {
            for (int j = 0; j < cols; ++j) {
                inputFile >> h_A[i * cols + j];
            }
        }

        // Read matrix2 elements
        for (int i = 0; i < cols; ++i) {

                inputFile >> h_B[i ];

        }

        // Print matrices for the current test case (you can replace this with your specific logic)
        std::cout << "Matrix1:" << std::endl;
        for (int i = 0; i < rows; ++i) {
            for (int j = 0; j < cols; ++j) {
                std::cout << h_A[i * cols + j] << " ";
            }
            std::cout << std::endl;
        }

        std::cout << "Matrix2:" << std::endl;
        for (int i = 0; i < cols; ++i) {
            for (int j = 0; j < 1; ++j) {
                std::cout << h_B[i * cols + j] << " ";
            }
            std::cout << std::endl;
        }

      // Copy data from host to device
    cudaMemcpy(d_A, h_A, rows * cols * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, h_B, cols * 1 * sizeof(float), cudaMemcpyHostToDevice);



    //...
    // Kernel invocation with one block of N * N * 1 threads
    dim3 threadsPerBlock(32, 32);
    dim3 numBlocks((cols-1) / threadsPerBlock.x+1,(rows-1) / threadsPerBlock.y+1);
    MatMul<<<numBlocks, threadsPerBlock>>>(d_A, d_B, d_C,rows,cols);




    //...
    // Copy the result matrix C from device to host
    cudaMemcpy(h_C, d_C, rows * 1 * sizeof(float), cudaMemcpyDeviceToHost);

    printf("h_C[0] = %f\n", h_C[0]);
    printf("PASSED\n");
    // Write matrices for the current test case to the output file

       for (int i = 0; i < rows; ++i) {

                outputFile << h_C[i]<<" ";

            outputFile << std::endl;
        }

    // Free device memory
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);

    // Free host memory
    free(h_A);
    free(h_B);
    free(h_C);

    }

    // Close the file
    inputFile.close();
    outputFile.close();

   return 0;
}


Overwriting Q2_2_31_2_33.cu


In [44]:
!chmod +x Q2_2_31_2_33.cu
!nvcc Q2_2_31_2_33.cu -o Q2
!./Q2 "Q2-sample-testcases-template-file.txt" "Q2-sample-output-file.txt"

Test Case 1:
2,2
Matrix1:
2.3 6.2 
2.6 5.4 
Matrix2:
1.1 
0 
h_C[0] = 17.410000
PASSED


In [45]:
!nvprof ./Q2  "Q2-sample-testcases-template-file.txt" "Q2-sample-output-file.txt"

Test Case 1:
2,2
==4309== NVPROF is profiling process 4309, command: ./Q2 Q2-sample-testcases-template-file.txt Q2-sample-output-file.txt
Matrix1:
2.3 6.2 
2.6 5.4 
Matrix2:
1.1 
0 
h_C[0] = 17.410000
PASSED
==4309== Profiling application: ./Q2 Q2-sample-testcases-template-file.txt Q2-sample-output-file.txt
==4309== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   48.95%  3.7440us         1  3.7440us  3.7440us  3.7440us  MatMul(float*, float*, float*, int, int)
                   27.61%  2.1120us         1  2.1120us  2.1120us  2.1120us  [CUDA memcpy DtoH]
                   23.44%  1.7930us         2     896ns     673ns  1.1200us  [CUDA memcpy HtoD]
      API calls:   99.67%  165.61ms         3  55.205ms  3.0430us  165.60ms  cudaMalloc
                    0.12%  200.99us         1  200.99us  200.99us  200.99us  cudaLaunchKernel
                    0.08%  138.25us       114  1.2120us     139ns  56.325us  cuDeviceGetAt